In [1]:
# python notebook for Make Your Own Neural Network
# code for a 3-layer neural network, and code for learning the MNIST dataset
# xytywh, 2019/06/18

In [2]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

In [3]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        # 这里使用sigmoid函数不太对，对于多分类问题，并且0-9每一类是互斥的，其实应该使用softmax，
        # 不过为了方便起见，就使用sigmoid，不会有太大影响
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    # query the neural network
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [4]:
# number of input, hidden and output nodes
#输入层784=28*28个结点
input_nodes = 784
#隐藏层200个结点
hidden_nodes = 200
#输出层10个结点，刚好对应10类(0 - 9)
output_nodes = 10

# learning rate
#学习率是一个超参数，试着更改学习率，看看对测试结果有何影响
#当然，由于测试数据集数量很少(10条数据)，可能效果不是很明显
learning_rate = 0.1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [ ]:
# load the mnist training data CSV file into a list
training_data_file = open("mnist_dataset/mnist_train_60000.csv", 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [ ]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 5

for e in range(epochs):
    # go through all records in the training data set
    print("Starting epoch-{}".format(e + 1))
    i = 0
    # 这里用的是随机梯度下降
    # batchsize为1，即每次训练只使用一个数据，共60000个训练数据，所以一个epoch共训练(更新参数)60000次
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        i += 1
        print("Iteration-{} the shape of inputs is:{}".format(i, inputs.shape))
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    print("Epoch-{} is finished\n".format(e + 1))
    pass


Starting epoch-1
Iteration-1 the shape of inputs is:(784,)
Iteration-2 the shape of inputs is:(784,)
Iteration-3 the shape of inputs is:(784,)
Iteration-4 the shape of inputs is:(784,)
Iteration-5 the shape of inputs is:(784,)
Iteration-6 the shape of inputs is:(784,)
Iteration-7 the shape of inputs is:(784,)
Iteration-8 the shape of inputs is:(784,)
Iteration-9 the shape of inputs is:(784,)
Iteration-10 the shape of inputs is:(784,)
Iteration-11 the shape of inputs is:(784,)
Iteration-12 the shape of inputs is:(784,)
Iteration-13 the shape of inputs is:(784,)
Iteration-14 the shape of inputs is:(784,)
Iteration-15 the shape of inputs is:(784,)
Iteration-16 the shape of inputs is:(784,)
Iteration-17 the shape of inputs is:(784,)
Iteration-18 the shape of inputs is:(784,)
Iteration-19 the shape of inputs is:(784,)
Iteration-20 the shape of inputs is:(784,)
Iteration-21 the shape of inputs is:(784,)
Iteration-22 the shape of inputs is:(784,)
Iteration-23 the shape of inputs is:(784,)
Ite

In [10]:
# load the mnist test data CSV file into a list
test_data_file = open("mnist_dataset/mnist_test_10000.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [11]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    print(correct_label, "correct label")
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    print(label, "network's answer")
    print(inputs.shape,label,correct_label,end="\n\n")
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

7 correct label
7 network's answer
(784,) 7 7

2 correct label
2 network's answer
(784,) 2 2

1 correct label
1 network's answer
(784,) 1 1

0 correct label
0 network's answer
(784,) 0 0

4 correct label
4 network's answer
(784,) 4 4

1 correct label
1 network's answer
(784,) 1 1

4 correct label
4 network's answer
(784,) 4 4

9 correct label
9 network's answer
(784,) 9 9

5 correct label
5 network's answer
(784,) 5 5

9 correct label
9 network's answer
(784,) 9 9

0 correct label
0 network's answer
(784,) 0 0

6 correct label
6 network's answer
(784,) 6 6

9 correct label
9 network's answer
(784,) 9 9

0 correct label
0 network's answer
(784,) 0 0

1 correct label
1 network's answer
(784,) 1 1

5 correct label
5 network's answer
(784,) 5 5

9 correct label
9 network's answer
(784,) 9 9

7 correct label
7 network's answer
(784,) 7 7

3 correct label
3 network's answer
(784,) 3 3

4 correct label
4 network's answer
(784,) 4 4

9 correct label
9 network's answer
(784,) 9 9

6 correct lab

In [12]:
# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("accuracy = ", scorecard_array.sum() / scorecard_array.size*100,"%")

accuracy =  97.43 %
